In [ ]:
#!conda create -n ag python=3.10
#!conda activate ag
#!conda install -c conda-forge mamba
#!mamba install -c conda-forge -c pytorch -c nvidia autogluon "pytorch=*=*cuda*"
#!mamba install -c conda-forge "ray-tune >=2.6.3,<2.7" "ray-default >=2.6.3,<2.7"  # install ray for faster training

In [4]:
# %pip install autogluon
#!pip install --upgrade numpy pandas scipy
#!pip install numpy==1.26.4
#!pip install pyJoules
#!pip install mxnet-cu110
#!pip install jedi
#!pip install setuptools
#!pip install scikit-learn==1.3.0
#!pip install pandas==2.0.0
#!pip install fsspec==2023.1.0
#!pip install torch==2.0.1+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#cls
# !pip install cudatoolkit


Defaulting to user installation because normal site-packages is not writeable
  Using cached autogluon-1.2-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.core-1.2-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon.features-1.2-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.tabular-1.2-py3-none-any.whl.metadata (14 kB)
  Using cached autogluon.multimodal-1.2-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon.timeseries-1.2-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon.common-1.2-py3-none-any.whl.metadata (11 kB)
  Using cached ray-2.39.0-cp312-cp312-win_amd64.whl.metadata (18 kB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached lightning-2.5.0.post0-py3-none-any.whl.metadata (40 kB)
     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ---------------------------------------- 44.4/44.4 kB 1.1 MB/s eta 0:00:00
  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
  

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [1]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.device_count())  # Should be > 0

True
1


In [2]:
import pandas as pd
import numpy as np
import logging
import os
import shutil
import time
import json
from sklearn.feature_selection import mutual_info_classif, VarianceThreshold
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Deep Learning Part

In [ ]:
FILENAME = "XAPI"
DATA_PATH = "xapi.csv"
TARGET = "Class"
KFOLD = 10  # Number of folds for cross-validation

# Load dataset
df = pd.read_csv(DATA_PATH)
# Remove spaces in column names (if any)
df.columns = df.columns.str.replace(' ', '')

# Separate features and target variable
X = df.drop(columns=[TARGET])
y = df[TARGET]

# Create a DataFrame including features and target
df_selected = X.copy()
df_selected[TARGET] = y

# Check if CUDA (GPU) is available
gpu_available = 1 if torch.cuda.is_available() else 0

validation_type = 'kfold'
    
start_time = time.time()

# Define output directory
path = f"GPU_{gpu_available}_{FILENAME}_DL_VALIDATION_{validation_type}"
# If the directory already exists, delete it before creating a new one
if os.path.exists(path):
    shutil.rmtree(path)  # Remove previous results
#os.makedirs(path, exist_ok=True)

# Create AutoGluon predictor
predictor = TabularPredictor(
    label=TARGET, 
    path=path, 
    problem_type="multiclass",
)

# Fit models using AutoGluon with 10-fold bagging
if gpu_available:
    predictor.fit(
        df_selected,
        num_bag_folds=KFOLD,
        verbosity=2,
        num_gpus=1,
        excluded_model_types=['RF', 'KNN', 'GBM', 'XGB', 'CAT', 'XT', 'LR'],
        presets="best_quality"
    )
else:
    predictor.fit(
        df_selected,
        num_bag_folds=KFOLD,
        verbosity=2,
        excluded_model_types=['RF', 'KNN', 'GBM', 'XGB', 'CAT', 'XT', 'LR'],
        presets="best_quality"
    )

####
# (A) Retrieve Out-Of-Fold (OOF) predictions
####
# This DataFrame contains predictions made for each instance when it was in the validation set of a given fold
oof_pred = predictor.get_oof_pred()

# Retrieve true labels corresponding to OOF predictions
y_true_oof = df_selected.loc[oof_pred.index, TARGET]

# Compute OOF metrics
accuracy_oof = accuracy_score(y_true_oof, oof_pred)
precision_oof = precision_score(y_true_oof, oof_pred, average='weighted')
recall_oof = recall_score(y_true_oof, oof_pred, average='weighted')
f1_oof = f1_score(y_true_oof, oof_pred, average='weighted')

# For multi-class ROC-AUC:
# predictor.get_oof_pred_proba() -> Then compute roc_auc_score
proba_oof = predictor.get_oof_pred_proba()
one_hot_true = pd.get_dummies(y_true_oof, drop_first=False)
roc_auc_oof = roc_auc_score(one_hot_true, proba_oof, average='weighted', multi_class='ovr')


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.4
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          32
Memory Avail:       50.21 GB / 63.94 GB (78.5%)
Disk Space Avail:   128.36 GB / 464.91 GB (27.6%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=10, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Running DyStack for u

AttributeError: 'TabularPredictor' object has no attribute 'get_cross_validation_scores'

In [5]:
# (B) Summary of cross-validation metrics
cv_scores = predictor.leaderboard(extra_info=True)

# Store OOF metrics in a DataFrame
df_oof_metrics = pd.DataFrame({
    'Metric': ['Accuracy (OOF)', 'Precision (OOF)', 'Recall (OOF)', 'F1 Score (OOF)', 'ROC AUC (OOF)'],
    'Score': [accuracy_oof, precision_oof, recall_oof, f1_oof, roc_auc_oof]
})

####
# (C) Evaluate model on the full training dataset (not a true test set)
####
y_pred_final = predictor.predict(df_selected)
y_prob_final = predictor.predict_proba(df_selected)

accuracy_final = accuracy_score(df_selected[TARGET], y_pred_final)
precision_final = precision_score(df_selected[TARGET], y_pred_final, average='weighted')
recall_final = recall_score(df_selected[TARGET], y_pred_final, average='weighted')
f1_final = f1_score(df_selected[TARGET], y_pred_final, average='weighted')

# For multi-class ROC-AUC:
roc_auc_final = roc_auc_score(pd.get_dummies(df_selected[TARGET]), 
                                y_prob_final, 
                                average='weighted', 
                                multi_class='ovr')

# Store final evaluation metrics in a DataFrame
df_final_metrics = pd.DataFrame({
    'Metric': ['Accuracy (Final)', 'Precision (Final)', 'Recall (Final)', 'F1 Score (Final)', 'ROC AUC (Final)'],
    'Score': [accuracy_final, precision_final, recall_final, f1_final, roc_auc_final]
})

# Save results to CSV files
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60

filename_oof = f"GPU_{gpu_available}_{FILENAME}_OOF_RESULTS_{validation_type}_{execution_time_minutes:.2f}.csv"
filename_final = f"GPU_{gpu_available}_{FILENAME}_FINAL_RESULTS_{validation_type}_{execution_time_minutes:.2f}.csv"

df_oof_metrics.to_csv(filename_oof, index=False)
df_final_metrics.to_csv(filename_final, index=False)

print("OOF Metrics saved to:", filename_oof)
print("Final Metrics saved to:", filename_final)
print("CV Scores summary:", cv_scores)
print("Execution time (min):", execution_time_minutes)

OOF Metrics saved to: GPU_1_XAPI_OOF_RESULTS_kfold_66.98.csv
Final Metrics saved to: GPU_1_XAPI_FINAL_RESULTS_kfold_66.98.csv
CV Scores summary:                           model  score_val eval_metric  pred_time_val  \
0           WeightedEnsemble_L3   0.885417    accuracy       3.396184   
1   NeuralNetFastAI_r134_BAG_L2   0.881250    accuracy       3.219157   
2   NeuralNetFastAI_r187_BAG_L2   0.879167    accuracy       3.219671   
3    NeuralNetFastAI_r65_BAG_L2   0.877083    accuracy       3.217616   
4   NeuralNetFastAI_r100_BAG_L2   0.877083    accuracy       3.234588   
..                          ...        ...         ...            ...   
85  NeuralNetFastAI_r191_BAG_L1   0.789583    accuracy       0.085309   
86  NeuralNetFastAI_r111_BAG_L1   0.785417    accuracy       0.094197   
87  NeuralNetFastAI_r160_BAG_L1   0.783333    accuracy       0.085326   
88  NeuralNetFastAI_r100_BAG_L1   0.779167    accuracy       0.091905   
89  NeuralNetFastAI_r194_BAG_L1   0.775000    accura

In [8]:
# ==========================
# MODEL INSPECTION SCRIPT
# ==========================

# This script retrieves information about trained AutoGluon models, including:
# Listing all trained models.
# Extracting hyperparameters from a specific model.
# Getting detailed training information.
# Saving hyperparameters to a JSON file.
# Displaying a full model summary.

# ==========================
# 1. List all trained models
# ==========================
# AutoGluon trains multiple models (e.g., stacked, bagged, ensembles).
# This command lists all trained models in the predictor.
model_names = predictor.get_model_names()
print("Available Models:", model_names)
# Example output: 
# ['WeightedEnsemble_L2', 
# 'LightGBMXT_BAG_L1', 
# 'LightGBM_BAG_L1', 
# 'NeuralNetMXNet_BAG_L1']

Available Models: ['NeuralNetFastAI_BAG_L1', 'NeuralNetTorch_BAG_L1', 'NeuralNetTorch_r79_BAG_L1', 'NeuralNetFastAI_r191_BAG_L1', 'NeuralNetTorch_r22_BAG_L1', 'NeuralNetFastAI_r102_BAG_L1', 'NeuralNetFastAI_r145_BAG_L1', 'NeuralNetTorch_r30_BAG_L1', 'NeuralNetTorch_r86_BAG_L1', 'NeuralNetFastAI_r11_BAG_L1', 'NeuralNetFastAI_r103_BAG_L1', 'NeuralNetTorch_r14_BAG_L1', 'NeuralNetFastAI_r143_BAG_L1', 'NeuralNetFastAI_r156_BAG_L1', 'NeuralNetFastAI_r95_BAG_L1', 'NeuralNetTorch_r41_BAG_L1', 'NeuralNetTorch_r158_BAG_L1', 'NeuralNetFastAI_r37_BAG_L1', 'NeuralNetTorch_r197_BAG_L1', 'NeuralNetFastAI_r134_BAG_L1', 'NeuralNetTorch_r143_BAG_L1', 'NeuralNetFastAI_r111_BAG_L1', 'NeuralNetTorch_r31_BAG_L1', 'NeuralNetFastAI_r65_BAG_L1', 'NeuralNetFastAI_r88_BAG_L1', 'NeuralNetTorch_r87_BAG_L1', 'NeuralNetTorch_r71_BAG_L1', 'NeuralNetTorch_r185_BAG_L1', 'NeuralNetFastAI_r160_BAG_L1', 'NeuralNetFastAI_r69_BAG_L1', 'NeuralNetFastAI_r138_BAG_L1', 'NeuralNetFastAI_r172_BAG_L1', 'NeuralNetTorch_r76_BAG_L1',

C:\Users\JAL\AppData\Local\Temp\ipykernel_34308\3445219386.py:19: DeprecationWarning: `get_model_names` has been deprecated and will be removed in version 1.2. Please use `model_names` instead. This will raise an error in the future!
  model_names = predictor.get_model_names()


In [ ]:
# ==========================
# 2. Get hyperparameters of a specific model
# ==========================
best_model_name = predictor.get_model_best()  # Get the name of the best-performing model
print(f"Best model: {best_model_name}")

# Retrieve hyperparameters of the best-performing model
model_info = predictor.info()
best_model_hyperparameters = model_info['model_info'][best_model_name]['hyperparameters']
print(f"Hyperparameters of the best model: {best_model_hyperparameters}")
#Example output: Hyperparameters of the best model:
# {'use_orig_features': False, 
# 'max_base_models': 25, 
# 'max_base_models_per_type': 5, 
# 'save_bag_folds': True}

C:\Users\JAL\AppData\Local\Temp\ipykernel_34308\2089814955.py:4: DeprecationWarning: `get_model_best` has been deprecated and will be removed in version 1.2. Please use `model_best` instead. This will raise an error in the future!
  best_model_name = predictor.get_model_best()  # Get the name of the best-performing model


Best model: WeightedEnsemble_L3
Hyperparameters of the best model: {'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}


In [ ]:
# ==========================
# Get model-specific training details
# ==========================
# Retrieve additional information about the model (training time, memory usage, etc.)
model_name = best_model_name  # or specify any other model name
model_info = predictor.info()
training_details = model_info['model_info'][model_name]
print(f"Training details of {model_name}:")
print(training_details)

# This may include:
# - Training time
# - Memory usage
# - Number of layers
# - Loss function used
# - Epochs trained
# Example output:
# {'name': 'WeightedEnsemble_L3', 'model_type': 'WeightedEnsembleModel', 'problem_type': 'multiclass', 'eval_metric': 'accuracy', 'stopping_metric': 'accuracy', 'fit_time': 0.07663822174072266, 'num_classes': 3, 'quantile_levels': None, 'predict_time': 0.0010030269622802734, 'val_score': 0.8854166666666666, 'hyperparameters': {'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}, 'hyperparameters_fit': {}, 'hyperparameters_nondefault': ['save_bag_folds'], 'ag_args_fit': {'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}, 'num_features': 6, 'features': ['NeuralNetTorch_r30_BAG_L2_1', 'NeuralNetFastAI_r134_BAG_L2_2', 'NeuralNetTorch_r30_BAG_L2_2', 'NeuralNetFastAI_r134_BAG_L2_1', 'NeuralNetTorch_r30_BAG_L2_0', 'NeuralNetFastAI_r134_BAG_L2_0'], 'feature_metadata': <autogluon.common.features.feature_metadata.FeatureMetadata object at 0x00000199481A5A10>, 'memory_size': 26017, 'compile_time': None, 'is_initialized': True, 'is_fit': True, 'is_valid': True, 'can_infer': True, 'bagged_info': {'child_model_type': 'GreedyWeightedEnsembleModel', 'num_child_models': 1, 'child_model_names': ['S1F1'], '_n_repeats': 1, '_k_per_n_repeat': [1], '_random_state': 3, 'low_memory': False, 'bagged_mode': False, 'max_memory_size': 26017, 'min_memory_size': 26017, 'child_hyperparameters': {'ensemble_size': 25, 'subsample_size': 1000000}, 'child_hyperparameters_fit': {'ensemble_size': 4}, 'child_ag_args_fit': {'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}}, 'stacker_info': {'num_base_models': 2, 'base_model_names': ['NeuralNetTorch_r30_BAG_L2', 'NeuralNetFastAI_r134_BAG_L2']}, 'children_info': {'S1F1': {'name': 'S1F1', 'model_type': 'GreedyWeightedEnsembleModel', 'problem_type': 'multiclass', 'eval_metric': 'accuracy', 'stopping_metric': 'accuracy', 'fit_time': 0.07663822174072266, 'num_classes': 3, 'quantile_levels': None, 'predict_time': None, 'val_score': None, 'hyperparameters': {'ensemble_size': 25, 'subsample_size': 1000000}, 'hyperparameters_fit': {'ensemble_size': 4}, 'hyperparameters_nondefault': [], 'ag_args_fit': {'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}, 'num_features': 6, 'features': ['NeuralNetTorch_r30_BAG_L2_0', 'NeuralNetTorch_r30_BAG_L2_1', 'NeuralNetTorch_r30_BAG_L2_2', 'NeuralNetFastAI_r134_BAG_L2_0', 'NeuralNetFastAI_r134_BAG_L2_1', 'NeuralNetFastAI_r134_BAG_L2_2'], 'feature_metadata': <autogluon.common.features.feature_metadata.FeatureMetadata object at 0x00000199481A7690>, 'memory_size': 7343, 'compile_time': None, 'is_initialized': True, 'is_fit': True, 'is_valid': True, 'can_infer': True, 'model_weights': {'NeuralNetTorch_r30_BAG_L2': 0.25, 'NeuralNetFastAI_r134_BAG_L2': 0.75}}}}

Training details of WeightedEnsemble_L3:
{'name': 'WeightedEnsemble_L3', 'model_type': 'WeightedEnsembleModel', 'problem_type': 'multiclass', 'eval_metric': 'accuracy', 'stopping_metric': 'accuracy', 'fit_time': 0.07663822174072266, 'num_classes': 3, 'quantile_levels': None, 'predict_time': 0.0010030269622802734, 'val_score': 0.8854166666666666, 'hyperparameters': {'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}, 'hyperparameters_fit': {}, 'hyperparameters_nondefault': ['save_bag_folds'], 'ag_args_fit': {'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}, 'num_features': 6, 'features': ['NeuralNetTorch_r30_BAG_L2_1

In [25]:
# ==========================
# Save model details to a JSON File
# ==========================
# Store hyperparameters in a structured JSON file for further analysis.
json_filename = f"{model_name}_hyperparams.json"
with open(json_filename, "w") as f:
    json.dump(best_model_hyperparameters, f, indent=4)

print(f"Model hyperparameters saved to {json_filename}")


Model hyperparameters saved to WeightedEnsemble_L3_hyperparams.json


In [26]:
# ==========================
# Retrieve full model summary
# ==========================
# Display a leaderboard with all trained models, including:
# - Train time
# - Validation accuracy
# - Performance metrics
summary = predictor.leaderboard(extra_info=True)
print("Full model summary:")
print(summary)

Full model summary:
                          model  score_val eval_metric  pred_time_val  \
0           WeightedEnsemble_L3   0.885417    accuracy       3.396184   
1   NeuralNetFastAI_r134_BAG_L2   0.881250    accuracy       3.219157   
2   NeuralNetFastAI_r187_BAG_L2   0.879167    accuracy       3.219671   
3    NeuralNetFastAI_r65_BAG_L2   0.877083    accuracy       3.217616   
4   NeuralNetFastAI_r100_BAG_L2   0.877083    accuracy       3.234588   
..                          ...        ...         ...            ...   
85  NeuralNetFastAI_r191_BAG_L1   0.789583    accuracy       0.085309   
86  NeuralNetFastAI_r111_BAG_L1   0.785417    accuracy       0.094197   
87  NeuralNetFastAI_r160_BAG_L1   0.783333    accuracy       0.085326   
88  NeuralNetFastAI_r100_BAG_L1   0.779167    accuracy       0.091905   
89  NeuralNetFastAI_r194_BAG_L1   0.775000    accuracy       0.078973   

       fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  \
0   1517.443671             